In [15]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Define your image and annotation directories
image_dir = 'C:/Users/baldo/OneDrive - MSFT/Desktop/Deep Learning/Pot Hole Detection/Natural Pothole Dataset within River Environments/Dataset'
annotation_dir = 'C:/Users/baldo/OneDrive - MSFT/Desktop/Deep Learning/Pot Hole Detection/Natural Pothole Dataset within River Environments/YOLO'

# Load annotations (assuming each annotation file contains multiple annotations per line)
def load_annotations(annotation_dir):
    annotations = {}
    for filename in os.listdir(annotation_dir):
        if filename.endswith('.txt'):
            image_name = os.path.splitext(filename)[0]  # Remove file extension
            with open(os.path.join(annotation_dir, filename), 'r') as f:
                lines = f.readlines()
                # Assuming multiple annotations per image (each line has class label and bounding box)
                for line in lines:
                    parts = line.strip().split()
                    class_label = int(parts[0])
                    x, y, width, height = map(float, parts[1:])
                    annotations.setdefault(image_name, []).append({
                        'class_label': class_label,
                        'bbox': (x, y, width, height)
                    })
    return annotations

# Load annotations from your annotation folder
annotations = load_annotations(annotation_dir)


# Load annotations (assuming each annotation file contains one class label per line)
annotations = load_annotations(annotation_dir)

# Create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
batch_size = 16
target_size = (128, 128)  # Adjust as needed

train_gen = datagen.flow_from_directory(
    image_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True,
    seed=42
)

val_gen = datagen.flow_from_directory(
    image_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

# Load pre-trained models
vgg_model = VGG16(weights='imagenet', include_top=False)
resnet_model = ResNet50(weights='imagenet', include_top=False)
effnet_model = EfficientNetB3(weights='imagenet', include_top=False)

# Add custom classification head
def build_custom_model(base_model):
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

vgg_custom_model = build_custom_model(vgg_model)
resnet_custom_model = build_custom_model(resnet_model)
effnet_custom_model = build_custom_model(effnet_model)

# Compile models
for model in [vgg_custom_model, resnet_custom_model, effnet_custom_model]:
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Evaluate models
for model, model_name in zip([vgg_custom_model, resnet_custom_model, effnet_custom_model],
                             ['VGG16', 'ResNet50', 'EfficientNetV2B3']):
    print(f"Model: {model_name}")
    y_true = val_gen.classes
    y_pred = model.predict(val_gen).flatten()
    y_pred_binary = np.round(y_pred)
    print(classification_report(y_true, y_pred_binary))

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred_binary)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title(f'Confusion Matrix ({model_name})')
    plt.show()

Found 2634 images belonging to 1 classes.
Found 658 images belonging to 1 classes.


In [ ]:
# Train models
for model in [vgg_custom_model, resnet_custom_model, effnet_custom_model]:
    model.fit(train_gen, validation_data=val_gen, epochs=10)

# Evaluate models
for model, model_name in zip([vgg_custom_model, resnet_custom_model, effnet_custom_model],
                             ['VGG16', 'ResNet50', 'EfficientNetV2B3']):
    print(f"Model: {model_name}")
    y_true = val_gen.classes
    y_pred = model.predict(val_gen).flatten()
    y_pred_binary = np.round(y_pred)
    print(classification_report(y_true, y_pred_binary))

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred_binary)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title(f'Confusion Matrix ({model_name})')
    plt.show()
